Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [30]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Folder dir for saving and loading files

In [31]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
# model_name = 'mtm_spring'
model_name = 'mtm_2spring'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [32]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_2spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s, -q4]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

mtm_2spring
['Coulomb', 'viscous', 'offset']


In [33]:
robot_def.bary_params

## Create kinematics chain

In [34]:
geom = Geometry(robot_def)

In [35]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [36]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/10
Calculating the link kinetic energy of 2/10
Calculating the link kinetic energy of 3/10
Calculating the link kinetic energy of 4/10
Calculating the link kinetic energy of 5/10
Calculating the link kinetic energy of 6/10
Calculating the link kinetic energy of 7/10
Calculating the link kinetic energy of 8/10
Calculating the link kinetic energy of 9/10
Calculating joint torques...
tau of q1
tau of q2
tau of q8
tau of q4
tau of q5
tau of q6
tau of q7
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q8, q4, q5, q6, q7, dq1, dq2, dq8, dq4, dq5, dq6, dq7, ddq1, ddq2, ddq8, ddq4, ddq5, ddq6, ddq7))
Calculating M, C and G...
Calculating base parameter...
[ 79  80  82  66  69  67  53  83  85  78  84  71  72  56  81  58  59  54
  40  45  46  65  47  41  43 110  32 111  57  14  55  92  44  37  30  39
  17  28  15  16  42  97  98  63  76  29  89  50  18 121  11  24 102  38
   

In [37]:
robot_def.bary_params

In [38]:
sympy.Matrix(dyn.base_param)

⎡                                                                  1.0⋅L7xy   
⎢                                                                             
⎢                                                                  1.0⋅L7xz   
⎢                                                                             
⎢                                                                  1.0⋅L7yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xy   
⎢                                                                             
⎢                                                                  1.0⋅L6yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xz   
⎢                                                                             
⎢                                                   

In [39]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

Saved 71 parameters


---
# 3 - Excitation trajectory optimization

In [40]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
#model_name = 'mtm_spring'
model_name = 'mtm_2spring'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [41]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [42]:
trajectory_name = 'one'
# trajectory_name = 'two_normalized'

In [44]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm' or model_name == 'mtm_spring' or model_name == 'mtm_2spring':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 8
cartesian constraint number: 0
constraint number: 32
frames_constrained: []
cond: 85.58123405
cond: 85.58123405
cond: 85.5812286922
cond: 85.581234943
cond: 85.5812414873
cond: 85.581236377
cond: 85.5812348116
cond: 85.5812386745
cond: 85.5812350053
cond: 85.5812446981
cond: 85.5812410134
cond: 85.5812309975
cond: 85.5812427132
cond: 85.5812346268
cond: 85.5812051868
cond: 85.5812638249
cond: 85.5812531803
cond: 85.5812539996
cond: 85.5812496346
cond: 85.5812325013
cond: 85.5812371718
cond: 85.5812028291
cond: 85.5812291596
cond: 85.5812381842
cond: 85.581233719
cond: 85.5812397921
cond: 85.5812340278
cond: 85.581238004
cond: 85.5812050666
cond: 85.5812052304
cond: 85.5812211212
cond: 85.5812230117
cond: 85.5812255065
cond: 85.5812341265
cond: 85.5812412381
cond: 85.5812403268
cond: 85.5812277277
cond: 85.5812208604
cond: 85.5812238973
cond: 85.581232471
cond: 85.5812385864
cond: 85.5812365979
cond: 85.5812326195
cond: 85.5812387082
cond: 85.5812352342
cond: 

cond: 128.265319071
cond: 128.265108998
cond: 128.265120803
cond: 128.265111378
cond: 128.265082628
cond: 128.265100229
cond: 128.265093628
cond: 128.265090771
cond: 128.265088369
cond: 128.265088335
cond: 128.265089338
cond: 128.265088158
cond: 128.265093683
cond: 128.265097537
cond: 128.265081465
cond: 128.265095925
cond: 128.265089084
cond: 128.265090604
cond: 128.265091441
cond: 128.265096366
cond: 128.265094945
cond: 128.265093426
cond: 128.265090631
cond: 128.265093152
cond: 128.265092129
cond: 128.265092353
cond: 128.26508812
cond: 128.265087763
cond: 128.265090766
cond: 128.265086816
cond: 128.265092065
cond: 128.265088777
cond: 128.265088829
cond: 128.265089098
cond: 128.265090973
cond: 128.26508993
cond: 128.265089974
cond: 128.265090081
cond: 128.265089403
cond: 128.26508349
cond: 128.265089241
cond: 128.265090396
cond: 128.265087979
cond: 128.265090674
cond: 128.265090055
cond: 128.265089931
cond: 128.265092764
cond: 128.26508823
cond: 128.265089785
cond: 128.265089202
cond

cond: 348.707091659
cond: 348.707093502
cond: 348.707093587
cond: 348.707091794
cond: 348.707092864
cond: 348.707093575
cond: 348.707093283
cond: 348.707092943
cond: 348.707093514
cond: 348.707093627
cond: 348.707095898
cond: 348.707093788
cond: 348.707093238
cond: 348.707093442
cond: 348.707093368
cond: 316.447180493
cond: 316.447180493
cond: 316.447162688
cond: 316.44716935
cond: 316.447148696
cond: 316.447119058
cond: 316.44712651
cond: 316.447109922
cond: 316.447190245
cond: 316.447155312
cond: 316.447184327
cond: 316.447162219
cond: 316.447179776
cond: 316.447153887
cond: 316.447101192
cond: 316.447144546
cond: 316.44728049
cond: 316.447207351
cond: 316.447098461
cond: 316.447093553
cond: 316.447086863
cond: 316.446602034
cond: 316.446931034
cond: 316.447147338
cond: 316.44706286
cond: 316.447370631
cond: 316.447252064
cond: 316.447227824
cond: 316.447174081
cond: 316.447156474
cond: 316.447088378
cond: 316.447303735
cond: 316.447247717
cond: 316.447198713
cond: 316.447937099
cond

cond: 237.64089747
cond: 237.640891173
cond: 237.640830201
cond: 237.640845688
cond: 237.640798939
cond: 237.64085953
cond: 237.640924416
cond: 237.640840657
cond: 237.641485449
cond: 237.641051192
cond: 237.64098218
cond: 237.640884415
cond: 237.640788223
cond: 237.640860115
cond: 237.640855356
cond: 237.64086168
cond: 237.640860041
cond: 237.640854679
cond: 237.640855393
cond: 237.640857199
cond: 237.640854834
cond: 237.640857797
cond: 237.640850719
cond: 237.640854571
cond: 237.640853528
cond: 237.640858008
cond: 237.640854297
cond: 237.640857607
cond: 237.640853551
cond: 237.640854346
cond: 237.640853282
cond: 237.640854169
cond: 237.640855006
cond: 237.640855237
cond: 237.640864761
cond: 237.640851489
cond: 237.64085654
cond: 237.640855637
cond: 237.640854802
cond: 237.640855691
cond: 237.640855057
cond: 237.640858329
cond: 237.640855847
cond: 237.640854151
cond: 237.640854176
cond: 237.640853732
cond: 237.640856868
cond: 237.64085267
cond: 237.640858229
cond: 237.640853137
cond: 

cond: 228.01152703
cond: 228.011527661
cond: 228.011526976
cond: 228.011526519
cond: 228.011527101
cond: 228.011527251
cond: 228.011526432
cond: 228.011526917
cond: 228.011528309
cond: 228.011531353
cond: 228.011529832
cond: 228.01152564
cond: 228.011527527
cond: 228.011527597
cond: 228.011527421
cond: 228.011527138
cond: 228.011526913
cond: 228.011527524
cond: 228.011527117
cond: 228.011527176
cond: 228.011527007
cond: 228.011527007
cond: 228.011526903
cond: 228.011527444
cond: 228.011527043
cond: 228.011527044
cond: 228.011527234
cond: 228.011527211
cond: 240.82146428
cond: 231.456699411
cond: 227.130443541
cond: 227.130443541
cond: 227.130432556
cond: 227.13042244
cond: 227.130404032
cond: 227.130416065
cond: 227.130402859
cond: 227.130382892
cond: 227.130433144
cond: 227.130431479
cond: 227.13045203
cond: 227.130443162
cond: 227.130418787
cond: 227.130429731
cond: 227.130437178
cond: 227.130429076
cond: 227.130496826
cond: 227.130488565
cond: 227.130450795
cond: 227.130401906
cond:

...

cond: 216.998303177
cond: 216.998303103
cond: 216.998302946
cond: 216.998303106
cond: 216.998303227
cond: 216.998303193
cond: 216.998303071
cond: 216.998303369
cond: 216.998303198
cond: 216.998303136
cond: 216.99830333
cond: 216.998303128
cond: 216.998303229
cond: 226.673129932
cond: 216.011388079
cond: 216.011388079
cond: 216.011378237
cond: 216.011368505
cond: 216.011356077
cond: 216.011357793
cond: 216.011353288
cond: 216.011330504
cond: 216.011383742
cond: 216.01137087
cond: 216.011397469
cond: 216.011388528
cond: 216.011368831
cond: 216.011379031
cond: 216.011385837
cond: 216.011400563
cond: 216.011432656
cond: 216.011412787
cond: 216.011407463
cond: 216.011330842
cond: 216.011373152
cond: 216.011072329
cond: 216.011148509
cond: 216.011293601
cond: 216.011298481
cond: 216.011432051
cond: 216.011411161
cond: 216.011430942
cond: 216.011468422
cond: 216.011374839
cond: 216.011363992
cond: 216.011377403
cond: 216.011456717
cond: 216.011372691
cond: 216.011870065
cond: 216.011578321
co

cond: 214.042950794
cond: 214.042948139
cond: 214.043087961
cond: 214.043066202
cond: 214.043082284
cond: 214.043123728
cond: 214.04302446
cond: 214.043016578
cond: 214.043037992
cond: 214.043110007
cond: 214.043024722
cond: 214.04350628
cond: 214.043228666
cond: 214.043116563
cond: 214.043101919
cond: 214.042967444
cond: 214.043053495
cond: 214.043041284
cond: 214.043042245
cond: 214.0430408
cond: 214.043040989
cond: 214.043040681
cond: 214.043041444
cond: 214.043041538
cond: 214.043039648
cond: 214.043041098
cond: 214.043040808
cond: 214.043040471
cond: 214.043041157
cond: 214.043041319
cond: 214.043041604
cond: 214.043042324
cond: 214.043041068
cond: 214.043041546
cond: 214.043041519
cond: 214.043041079
cond: 214.043041424
cond: 214.043040204
cond: 214.043041335
cond: 214.043041077
cond: 214.043041133
cond: 214.043041203
cond: 214.043041085
cond: 214.043040829
cond: 214.043042927
cond: 214.043041433
cond: 214.043041275
cond: 214.043041251
cond: 214.043041034
cond: 214.043041755
cond

cond: 212.74268186
cond: 212.742681372
cond: 212.742681446
cond: 212.742681681
cond: 212.742681325
cond: 212.742681022
cond: 212.742682866
cond: 212.742682178
cond: 212.742681982
cond: 212.742681036
cond: 212.742680952
cond: 212.742681994
cond: 212.742681684
cond: 212.742681357
cond: 212.742680716
cond: 212.742681171
cond: 212.742681241
cond: 212.742682027
cond: 212.742681503
cond: 212.742681912
cond: 212.742681274
cond: 212.742681601
cond: 212.742681425
cond: 212.74268156
cond: 212.742681491
cond: 212.74268193
cond: 212.742681408
cond: 212.742681568
cond: 212.74268166
cond: 212.7426815
cond: 212.742681608
cond: 223.825347962
cond: 212.381751436
cond: 212.381751436
cond: 212.381741831
cond: 212.38173293
cond: 212.381716416
cond: 212.381723491
cond: 212.381719926
cond: 212.381694998
cond: 212.381747248
cond: 212.381733684
cond: 212.381764461
cond: 212.38175019
cond: 212.381732071
cond: 212.381739419
cond: 212.381738997
cond: 212.38176685
cond: 212.381799451
cond: 212.381765963
cond: 212

cond: 212.015729826
cond: 212.015726729
cond: 212.015702672
cond: 212.015754677
cond: 212.015740454
cond: 212.015772487
cond: 212.015756199
cond: 212.015738174
cond: 212.015746928
cond: 212.01574447
cond: 212.015777866
cond: 212.015806924
cond: 212.015771246
cond: 212.015776451
cond: 212.015711568
cond: 212.015747162
cond: 212.015460222
cond: 212.015537962
cond: 212.015672022
cond: 212.015666602
cond: 212.015810697
cond: 212.015784672
cond: 212.015794355
cond: 212.01583111
cond: 212.015744139
cond: 212.015738353
cond: 212.015755762
cond: 212.015819797
cond: 212.015742238
cond: 212.016205686
cond: 212.015938902
cond: 212.015829719
cond: 212.015821054
cond: 212.015679552
cond: 212.015770649
cond: 212.015758767
cond: 212.015761064
cond: 212.015758237
cond: 212.015758869
cond: 212.015758994
cond: 212.015758366
cond: 212.015757876
cond: 212.015758632
cond: 212.015758301
cond: 212.015759549
cond: 212.015759403
cond: 212.015759565
cond: 212.015758863
cond: 212.015758133
cond: 212.015759241
co

cond: 212.008942095
cond: 212.008933444
cond: 212.008791966
cond: 212.008883028
cond: 212.008871131
cond: 212.008873417
cond: 212.008870604
cond: 212.008871237
cond: 212.008871363
cond: 212.008870735
cond: 212.008870248
cond: 212.008871002
cond: 212.008870662
cond: 212.008871911
cond: 212.00887177
cond: 212.008871931
cond: 212.008871231
cond: 212.008870505
cond: 212.008871608
cond: 212.008870698
cond: 212.008871019
cond: 212.008871306
cond: 212.008871306
cond: 212.008871377
cond: 212.008871285
cond: 212.008870469
cond: 212.008870664
cond: 212.008870968
cond: 212.00887091
cond: 212.008870917
cond: 212.008870787
cond: 212.008872254
cond: 212.008870416
cond: 212.008871345
cond: 212.008870395
cond: 212.008871242
cond: 212.008870661
cond: 212.008869747
cond: 212.008870493
cond: 212.00887144
cond: 212.008871169
cond: 212.008870933
cond: 212.008870779
cond: 212.008871074
cond: 212.008871075
cond: 212.008871018
cond: 212.008871144
cond: 212.008870901
cond: 212.008871098
cond: 212.008871077
con

In [45]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [46]:
traj_optimizer.calc_frame_traj()

In [50]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [48]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [49]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.2305,0.0365,-0.0533,-0.0309,0.1137,0.0827,2.6281,-0.0007,0.0771,-0.1183,-0.0248,0.212,0.6409
q2,0.3439,0.0707,-0.1548,-0.1014,-0.2553,0.1854,0.387,0.0166,0.3863,0.051,0.8859,-0.1995,-0.9341
q8,0.308,-0.0115,-0.0084,0.0559,-0.1531,-0.3528,0.5565,-0.104,0.0024,-0.0558,0.2911,0.4162,-0.7182
q4,1.2666,-0.1077,0.0244,-0.1766,0.1829,0.6939,-1.7186,0.2642,1.0795,0.4964,-0.017,-2.3291,0.2692
q5,0.3422,-0.5249,-0.1741,-1.206,-0.9952,-0.3436,0.3237,-0.3805,-0.3327,0.5253,0.2745,0.2215,0.7684
q6,0.0342,-0.0772,0.0558,-0.0636,0.058,-0.1256,-0.5243,-0.0612,0.1768,0.0496,-0.291,-0.2854,-0.2442
q7,0.0344,-1.0205,1.2635,-3.1957,3.1248,5.571,0.5602,0.0419,-0.6761,2.1075,4.793,1.7796,9.7753


---
# Data processing

In [7]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

In [8]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
# model_name = 'mtm'
model_name = 'mtm_spring'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

# trajectory_name = 'one'
# trajectory_name = 'two'
trajectory_name = 'one'

In [9]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
dof, fourier_order, base_freq, traj_optimizer_result, reg_norm_mat = load_data(trajectory_folder, trajectory_name)


trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_data(model_folder,model_name)

(<type 'numpy.ndarray'>, (14989, 21))


## Calculate filter cut-off frequency

In [10]:
fc_mult = 4.0
# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 2.4


## Get velocity and acceleration data and filter raw data

In [11]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (14989, 7)
0
1
2
3
4
5
6
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [12]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [13]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((102123, 70), (102123,))
1272411.6409042422


In [14]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [15]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,-3.7606175214867072e-06
1.0*L7xz,-1.2450854463090131e-05
1.0*L7yz,-1.1660400486733329e-05
1.0*L6xy,0.0007151560568303215
1.0*L6xz,0.00015479673256722955
1.0*L6yz,-3.0292284948964937e-06
1.0*L6xx - 1.0*L6yy + 1.0*L7xx,-0.00014056381839007426
1.0*L7zz,-8.435077808115199e-06
1.0*l7x,0.00016686052118938116
1.0*l7y,0.000176728317155872


## Compare measured torque and predicted torque

### Generate predicted torque

In [16]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [17]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.00029406210815391344
standard deviation of xb using OLS:
[2.13065943e-06 7.77031141e-07 7.70964624e-07 1.19162010e-05
 1.18044808e-05 1.47165887e-05 2.58875236e-05 2.70527243e-06
 2.62807375e-06 2.61597920e-06 2.14457053e-05 2.48666125e-05
 3.41628215e-05 1.63086485e-05 1.36844101e-05 2.56460143e-05
 1.92633987e-05 3.86445381e-05 3.70108273e-05 3.88950367e-05
 3.20612698e-05 2.88127010e-05 3.02852537e-04 8.10187198e-04
 1.52278583e-04 3.21840104e-05 2.53185943e-03 2.73535714e-05
 2.46117949e-05 1.02659910e-04 2.63325732e-04 3.17097202e-04
 2.63551171e-04 3.55116336e-05 3.51681789e-04 2.63041936e-04
 4.16153528e-05 9.93450021e-05 6.32080953e-05 5.08550980e-04
 2.62237281e-04 8.12881681e-04 8.39408031e-04 5.31400679e-04
 2.35580722e-05 9.88154874e-05 1.05351783e-04 3.85986432e-04
 1.39755005e-04 2.62696300e-04 3.24876403e-04 4.58346937e-04
 3.26487179e-03 3.39236538e-04 1.42025436e-04 1.87593285e-04
 1.80192721e-04 1.50768580e-04 1.42375518e-04 2

### Plot measured torque and predicted torque

In [18]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])

## Weighted Least Square (WLS)

In [19]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.02260302, 0.04156412, 0.03030228, 0.0196861 , 0.01042662,
       0.01210039, 0.0033314 ])

In [20]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.60738506, 4.38921693, 3.05917211, 0.42470381, 0.17455165,
       0.10147109, 0.00596803])

In [21]:
W_b.shape, tau_s.shape

In [22]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [23]:
weights.shape

In [24]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [25]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [26]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                  1.0⋅L7xy   
⎢                                                                             
⎢                                                                  1.0⋅L7xz   
⎢                                                                             
⎢                                                                  1.0⋅L7yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xy   
⎢                                                                             
⎢                                                                  1.0⋅L6xz   
⎢                                                                             
⎢                                                                  1.0⋅L6yz   
⎢                                                                             
⎢                                                   

In [27]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [28]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

In [29]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.06793809, 0.26859411, 0.16367809, 0.34087756, 0.2453117 ,
       0.21977706, 0.21779594])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [149]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [150]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [151]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [173]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [174]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 120)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4268296, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
Variables n = 121, constraints m = 102447
Cones:	linear vars: 133
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.62e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.50

  9400| 3.10e-01  1.06e-02  5.74e-04  1.25e+02  1.26e+02  5.87e-14  1.63e+03 
  9500| 3.10e-01  1.06e-02  5.65e-04  1.27e+02  1.27e+02  5.98e-14  1.65e+03 
  9600| 3.09e-01  1.00e-02  5.54e-04  1.28e+02  1.28e+02  6.08e-14  1.67e+03 
  9700| 3.09e-01  1.02e-02  5.43e-04  1.29e+02  1.29e+02  6.19e-14  1.69e+03 
  9800| 3.08e-01  1.05e-02  5.35e-04  1.30e+02  1.30e+02  6.30e-14  1.70e+03 
  9900| 3.08e-01  9.87e-03  5.24e-04  1.31e+02  1.32e+02  6.40e-14  1.72e+03 
 10000| 3.08e-01  9.73e-03  5.15e-04  1.33e+02  1.33e+02  6.51e-14  1.74e+03 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate
Timing: Solve time: 1.74e+03s
	Lin-sys: avg # CG iterations: 35.15, avg solve time: 1.72e-01s
	Cones: avg projection time: 2.59e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 6.4829e-09, dist(y, K*) = 2.7388e-09, s'y/|s||y| = 2.0676e-11
|Ax + s -

## Compare measured torque and predicted torque

### Generate predicted torque

In [175]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [200]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [196]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    0.986192165056305  ⎤
⎢                           ⎥
⎢L1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢L1xz  -7.76343556708024e-27⎥
⎢                           ⎥
⎢L1yy    0.986192165056345  ⎥
⎢                           ⎥
⎢L1yz  -2.12133368993413e-16⎥
⎢                           ⎥
⎢L1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢l1x   2.40588966703881e-27 ⎥
⎢                           ⎥
⎢l1y   4.76358117877349e-17 ⎥
⎢                           ⎥
⎢l1z     0.144175828054554  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢L2xx   0.0194654973664062  ⎥
⎢                           ⎥
⎢L2xy   0.00876972571924054 ⎥
⎢                           ⎥
⎢L2xz   0.00593865075240333 ⎥
⎢                           ⎥
⎢L2yy   0.0330003696264969  ⎥
⎢         

In [178]:
from identification import barycentric2standard_params

In [179]:
sdp_opt_std.x_result

array([ 9.86192165e-01, -1.38516767e-14, -7.76343557e-27,  9.86192165e-01,
       -2.12133369e-16,  5.58233934e-05,  2.40588967e-27,  4.76358118e-17,
        1.44175828e-01,  1.79307212e+00,  4.19135562e-02,  4.97210098e-02,
       -6.98865409e-02,  1.94654974e-02,  8.76972572e-03,  5.93865075e-03,
        3.30003696e-02,  9.67284032e-04,  5.04838084e-02, -4.02232774e-01,
        6.80708808e-02,  6.43105023e-04,  5.00027870e+00,  1.09124988e-01,
        1.99956998e-01, -2.00003749e-01,  6.19401340e-04,  8.10870769e-04,
        6.31621227e-05,  1.18055051e-03, -3.76610295e-05,  1.71030714e-03,
        4.07898714e-02, -2.79045453e-02, -2.19072695e-03,  1.41404147e+00,
        1.00110498e-03,  9.95052980e-04,  2.00008574e-01,  2.02184077e-02,
        3.45713217e-03,  1.34006236e-03,  1.24674614e-02,  2.77597880e-03,
        1.57090952e-03, -8.13941900e-04,  1.04485762e-02, -1.29595950e-02,
        1.33525052e-01,  4.08774176e-02,  1.22874520e-02, -1.04736499e-04,
        7.97410096e-04, -

In [180]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [181]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    0.974599396882406  ⎤
⎢                           ⎥
⎢I1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢I1xz  -7.56998482337162e-27⎥
⎢                           ⎥
⎢I1yy    0.974599396882446  ⎥
⎢                           ⎥
⎢I1yz  -2.08303108884751e-16⎥
⎢                           ⎥
⎢I1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢r1x   1.34176960395484e-27 ⎥
⎢                           ⎥
⎢r1y   2.65665899779873e-17 ⎥
⎢                           ⎥
⎢r1z    0.0804071551405432  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢I2xx   0.0185387373447571  ⎥
⎢                           ⎥
⎢I2xy   0.00329396309376292 ⎥
⎢                           ⎥
⎢I2xz   0.00588691805246235 ⎥
⎢                           ⎥
⎢I2yy  0.000643849538560864 ⎥
⎢         

In [182]:
sdp_constraints